In [15]:
#Solve 2D Poisson: u_xx + u_yy = f(x,y), on the unit square with b.c. 
# u(0,y) = g3(y), u(1,y) = g4(y), -u_y(x,0) = g1(x), u_y(x,1) = g2(x)
# Take the exact solution u(x,y) = sin(pi*x + pi*y)

#Transfers to discretized system
#(D2x + D2y + P1 + P2 + P3 + P4)u = b + f where
#P1 = alpha1*Hyinv*E1*BySy
#P2 = alphas2*Hyinv*E2*BySy
#P3 = alpha3*Hxinv*E3 + beta*Hxinv*BxSx_tran*E3
#P4 = alpha4*Hxinv*E4 + beta*Hxinv*BxSx_tran*E4

#b = alpha1*Hyinv*E1*g1 + alpha2*Hyinv*E2*g2 + alpha3*Hxinv*E3*g3 + beta*Hxinv*BxSx_tran*E3*g3 + ...
#    alpha4*Hxinv*E4*g4 + beta*Hxinv*BxSx_tran*E4*g4

#to make system PD, multiply by -(H kron H):


include("deriv_ops.jl")
using SparseArrays
using LinearMaps
using IterativeSolvers
using Parameters



@with_kw struct variables 
    h = 0.05
    dx = h
    dy = h
    x = 0:dx:1
    y = 0:dy:1
    Nx = length(x)
    Ny = length(y)	
    alpha1 = -1
    alpha2 = -1
    alpha3 = -13/dy
    alpha4 = -13/dy
    beta = 1
end

var_test = variables()

function myMAT!(du::AbstractVector, u::AbstractVector,var_test::variables)
	#Chunk below should be passed as input, but for now needs to match chunk below
# 	h = 0.05 
# 	dx = h
# 	dy = h
# 	x = 0:dx:1
#         y = 0:dy:1
# 	Nx = length(x)
#         Ny = length(y)
# 	alpha1 = -1
#         alpha2 = -1
#         alpha3 = -13/dy
#         alpha4 = -13/dy
#         beta = 1
    @unpack h,dx,dy,x,y,Nx,Ny,alpha1,alpha2,alpha3,alpha3,beta = var_test
	########################################

        du_ops = D2x(u,Nx,Ny,dx) + D2y(u,Nx,Ny,dy) #compute action of D2x + D2y

        du1 = BySy(u,Nx,Ny,dy)
        du2 = VOLtoFACE(du1,1,Nx,Ny)
        du3 = alpha1*Hyinv(du2,Nx,Ny,dy)  #compute action of P1

        du4 = BySy(u,Nx,Ny,dy)
        du5 = VOLtoFACE(du4,2,Nx,Ny)
        du6 = alpha2*Hyinv(du5,Nx,Ny,dy) #compute action of P2

        du7 = VOLtoFACE(u,3,Nx,Ny)
        du8 = BxSx_tran(du7,Nx,Ny,dx)
        du9 = beta*Hxinv(du8,Nx,Ny,dx)
        du10 = VOLtoFACE(u,3,Nx,Ny)
        du11 = alpha3*Hxinv(du10,Nx,Ny,dx) #compute action of P3

        du12 = VOLtoFACE(u,4,Nx,Ny)
        du13 = BxSx_tran(du12,Nx,Ny,dx)
        du14 = beta*Hxinv(du13,Nx,Ny,dx)
        du15 = VOLtoFACE(u,4,Nx,Ny)
        du16 = alpha4*Hxinv(du15,Nx,Ny,dx) #compute action of P4


        du0 = du_ops + du3 + du6 + du9 + du11 + du14 + du16 #Collect together

        #compute action of -Hx kron Hy:

        du17 = Hy(du0, Nx, Ny, dy)
	du[:] = -Hx(du17,Nx,Ny,dx)
end



 @unpack h,dx,dy,x,y,Nx,Ny,alpha1,alpha2,alpha3,alpha3,beta = var_test

N = Nx*Ny
g1 = -pi .* cos.(pi .* x)
g2 = pi .* cos.(pi .* x .+ pi)
g3 = sin.(pi .* y)
g4 = sin.(pi .+ pi .* y)

f = spzeros(Nx,Ny)
exactU = spzeros(Nx,Ny)

for i = 1:Nx
	for j = 1:Ny
		f[j,i] = -pi^2 .* sin.(pi .* x[i] + pi .* y[j]) - pi^2 .* sin.(pi .* x[i] + pi .* y[j])
		exactU[j,i] = sin.(pi .* x[i] + pi .* y[j])
	end
end
        
f = f[:]
exact = exactU[:]

#Construct vector b
b0 = FACEtoVOL(g1,1,Nx,Ny)
b1 = alpha1*Hyinv(b0,Nx,Ny,dy)

b2 = FACEtoVOL(g2,2,Nx,Ny)
b3 = alpha2*Hyinv(b2,Nx,Ny,dy)

b4 = FACEtoVOL(g3,3,Nx,Ny)
b5 = alpha3*Hxinv(b4,Nx,Ny,dx)
b6 = BxSx_tran(b4,Nx,Ny,dx)
b7 = beta*Hxinv(b6,Nx,Ny,dx)

b8 = FACEtoVOL(g4,4,Nx,Ny)
b9 = alpha4*Hxinv(b8,Nx,Ny,dx)
b10 = BxSx_tran(b8,Nx,Ny,dx)
b11 = beta*Hxinv(b10,Nx,Ny,dx)
        
bb = b1  + b3  + b5 + b7 + b9 + b11 + f
        
#Modify b for PD system
b12 = Hx(bb,Nx,Ny,dx)
b = -Hy(b12,Nx,Ny,dy)

D = LinearMap(myMAT!, N; ismutating=true)
u = cg(D,b,tol=1e-14)

diff = u - exact

Hydiff = Hy(diff,Nx,Ny,dy)
HxHydiff = Hx(Hydiff,Nx,Ny,dx)

err = sqrt(diff'*HxHydiff)
       
@show err




err = 0.0011080009255538023


0.0011080009255538023

In [4]:
@with_kw struct variables 
    h = 0.05
    dx = h
    dy = h
    x = 0:dx:1
    y = 0:dy:1
    Nx = length(x)
    Ny = length(y)	
    alpha1 = -1
    alpha2 = -1
    alpha3 = -13/dy
    alpha4 = -13/dy
    beta = 1
end

variables

In [5]:
var_test = variables()

variables
  h: Float64 0.05
  dx: Float64 0.05
  dy: Float64 0.05
  x: StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
  y: StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
  Nx: Int64 21
  Ny: Int64 21
  alpha1: Int64 -1
  alpha2: Int64 -1
  alpha3: Float64 -260.0
  alpha4: Float64 -260.0
  beta: Int64 1


In [7]:
@unpack h,dx,dy,x,y,Nx,Ny,alpha1,alpha2,alpha3,alpha3,beta = var_test

variables
  h: Float64 0.05
  dx: Float64 0.05
  dy: Float64 0.05
  x: StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
  y: StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
  Nx: Int64 21
  Ny: Int64 21
  alpha1: Int64 -1
  alpha2: Int64 -1
  alpha3: Float64 -260.0
  alpha4: Float64 -260.0
  beta: Int64 1


0.05

In [10]:
function myMAT!(du::AbstractVector, u::AbstractVector,var_test::variables)
	#Chunk below should be passed as input, but for now needs to match chunk below
# 	h = 0.05 
# 	dx = h
# 	dy = h
# 	x = 0:dx:1
#         y = 0:dy:1
# 	Nx = length(x)
#         Ny = length(y)
# 	alpha1 = -1
#         alpha2 = -1
#         alpha3 = -13/dy
#         alpha4 = -13/dy
#         beta = 1
    @unpack h,dx,dy,x,y,Nx,Ny,alpha1,alpha2,alpha3,alpha3,beta = var_test
	########################################

        du_ops = D2x(u,Nx,Ny,dx) + D2y(u,Nx,Ny,dy) #compute action of D2x + D2y

        du1 = BySy(u,Nx,Ny,dy)
        du2 = VOLtoFACE(du1,1,Nx,Ny)
        du3 = alpha1*Hyinv(du2,Nx,Ny,dy)  #compute action of P1

        du4 = BySy(u,Nx,Ny,dy)
        du5 = VOLtoFACE(du4,2,Nx,Ny)
        du6 = alpha2*Hyinv(du5,Nx,Ny,dy) #compute action of P2

        du7 = VOLtoFACE(u,3,Nx,Ny)
        du8 = BxSx_tran(du7,Nx,Ny,dx)
        du9 = beta*Hxinv(du8,Nx,Ny,dx)
        du10 = VOLtoFACE(u,3,Nx,Ny)
        du11 = alpha3*Hxinv(du10,Nx,Ny,dx) #compute action of P3

        du12 = VOLtoFACE(u,4,Nx,Ny)
        du13 = BxSx_tran(du12,Nx,Ny,dx)
        du14 = beta*Hxinv(du13,Nx,Ny,dx)
        du15 = VOLtoFACE(u,4,Nx,Ny)
        du16 = alpha4*Hxinv(du15,Nx,Ny,dx) #compute action of P4


        du0 = du_ops + du3 + du6 + du9 + du11 + du14 + du16 #Collect together

        #compute action of -Hx kron Hy:

        du17 = Hy(du0, Nx, Ny, dy)
	du[:] = -Hx(du17,Nx,Ny,dx)
end

myMAT! (generic function with 2 methods)

In [ ]:
function myMAT!(du::AbstractVector, u::AbstractVector)
	#Chunk below should be passed as input, but for now needs to match chunk below
	h = 0.05 
	dx = h
	dy = h
	x = 0:dx:1
        y = 0:dy:1
	Nx = length(x)
        Ny = length(y)
	alpha1 = -1
        alpha2 = -1
        alpha3 = -13/dy
        alpha4 = -13/dy
        beta = 1
	########################################

        du_ops = D2x(u,Nx,Ny,dx) + D2y(u,Nx,Ny,dy) #compute action of D2x + D2y

        du1 = BySy(u,Nx,Ny,dy)
        du2 = VOLtoFACE(du1,1,Nx,Ny)
        du3 = alpha1*Hyinv(du2,Nx,Ny,dy)  #compute action of P1

        du4 = BySy(u,Nx,Ny,dy)
        du5 = VOLtoFACE(du4,2,Nx,Ny)
        du6 = alpha2*Hyinv(du5,Nx,Ny,dy) #compute action of P2

        du7 = VOLtoFACE(u,3,Nx,Ny)
        du8 = BxSx_tran(du7,Nx,Ny,dx)
        du9 = beta*Hxinv(du8,Nx,Ny,dx)
        du10 = VOLtoFACE(u,3,Nx,Ny)
        du11 = alpha3*Hxinv(du10,Nx,Ny,dx) #compute action of P3

        du12 = VOLtoFACE(u,4,Nx,Ny)
        du13 = BxSx_tran(du12,Nx,Ny,dx)
        du14 = beta*Hxinv(du13,Nx,Ny,dx)
        du15 = VOLtoFACE(u,4,Nx,Ny)
        du16 = alpha4*Hxinv(du15,Nx,Ny,dx) #compute action of P4


        du0 = du_ops + du3 + du6 + du9 + du11 + du14 + du16 #Collect together

        #compute action of -Hx kron Hy:

        du17 = Hy(du0, Nx, Ny, dy)
	du[:] = -Hx(du17,Nx,Ny,dx)
end
	

In [ ]:
#What's below should be passed as input to function above...
h = 0.05
dx = h
dy = h
x = 0:dx:1
y = 0:dy:1
Nx = length(x)
Ny = length(y)	
alpha1 = -1
alpha2 = -1
alpha3 = -13/dy
alpha4 = -13/dy
beta = 1
##########################################

LoadError: syntax: space required before colon in "?" expression

In [11]:
Int64:Float64

MethodError: MethodError: no method matching -(::Type{Float64}, ::Type{Int64})

In [14]:
a = Int64[1,2,3,4,5,6,7,8,9,10]

10-element Array{Int64,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [15]:
b = Number[1,2,3,4,5,6,7,8,9,10]

10-element Array{Number,1}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [16]:
function arr_sumsqr{T <: Number}(x::Array{T})
    r = zero(T)
    for i = 1:length(x)
        r = r + x[i]^2
    end
    return r
end

UndefVarError: UndefVarError: T not defined

In [18]:
?immutable

search: isimmutable

Couldn't find immutable
Perhaps you meant isimmutable or iswritable


No documentation found.

Binding `immutable` does not exist.


In [19]:
p = 2
function pow_array(x::Vector{Float64})
    s = 0.0
    for y in x
        s = s + y^p
    end
    return s
end

pow_array (generic function with 1 method)

In [23]:
pow_array(Float64[2,3])

13.0

In [24]:
t = rand(100000)

100000-element Array{Float64,1}:
 0.6542411720260983 
 0.0913397952784274 
 0.2085698459135461 
 0.9278574460743669 
 0.46940952819460646
 0.4832794044942048 
 0.8325829909185998 
 0.4458146083318648 
 0.7981543117739327 
 0.24625012936293578
 0.26245340613919677
 0.8870365011376304 
 0.09839049890761564
 ⋮                  
 0.7646765051493525 
 0.8704773281434093 
 0.8494578293337467 
 0.14527082424122129
 0.356869648032472  
 0.5517813157936655 
 0.9798405524278362 
 0.773090279202391  
 0.5019812153125178 
 0.07575047403773949
 0.23993605339303148
 0.8396072665017813 

In [29]:
import Pkg
Pkg.add("BenchmarkTools")

 Resolving package versions...
 Installed Missings ─ v0.4.2
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
  [e1d29d7a] ↑ Missings v0.4.1 ⇒ v0.4.2


In [31]:
using BenchmarkTools
@benchmark pow_array(t)

┌ Info: Recompiling stale cache file /Users/chern/.julia/compiled/v1.1/BenchmarkTools/ZXPQo.ji for BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1184


BenchmarkTools.Trial: 
  memory estimate:  4.58 MiB
  allocs estimate:  300000
  --------------
  minimum time:     7.474 ms (0.00% GC)
  median time:      9.554 ms (0.00% GC)
  mean time:        10.374 ms (4.83% GC)
  maximum time:     105.382 ms (90.54% GC)
  --------------
  samples:          481
  evals/sample:     1

In [32]:
@code_warntype pow_array(t)

Body::Any
1 ── %1  = (Base.arraylen)(x)::Int64
│    %2  = (Base.sle_int)(0, %1)::Bool
│    %3  = (Base.bitcast)(UInt64, %1)::UInt64
│    %4  = (Base.ult_int)(0x0000000000000000, %3)::Bool
│    %5  = (Base.and_int)(%2, %4)::Bool
└───       goto #3 if not %5
2 ── %7  = (Base.arrayref)(false, x, 1)::Float64
└───       goto #4
3 ──       goto #4
4 ┄─ %10 = φ (#2 => false, #3 => true)::Bool
│    %11 = φ (#2 => %7)::Float64
│    %12 = φ (#2 => 2)::Int64
└───       goto #5
5 ── %14 = (Base.not_int)(%10)::Bool
└───       goto #11 if not %14
6 ┄─ %16 = φ (#5 => 0.0, #10 => %20)::Any
│    %17 = φ (#5 => %11, #10 => %33)::Float64
│    %18 = φ (#5 => %12, #10 => %34)::Int64
│    %19 = (%17 ^ Main.p)::Any
│    %20 = (%16 + %19)::Any
│    %21 = (Base.bitcast)(UInt64, %18)::UInt64
│    %22 = (Base.sub_int)(%21, 0x0000000000000001)::UInt64
│    %23 = (Base.arraylen)(x)::Int64
│    %24 = (Base.sle_int)(0, %23)::Bool
│    %25 = (Base.bitcast)(UInt64, %23)::UInt64
│    %26 = (Base.ult_int)(%22, %25)::Boo

In [33]:
p = 2
function pow_array(x::Vector{Float64})
    s = 0.0
    for y in x
        s = s + y^p
    end
    return s
end

pow_array (generic function with 1 method)

In [34]:
@benchmark pow_array(t)

BenchmarkTools.Trial: 
  memory estimate:  4.58 MiB
  allocs estimate:  300000
  --------------
  minimum time:     7.048 ms (0.00% GC)
  median time:      7.508 ms (0.00% GC)
  mean time:        7.762 ms (4.02% GC)
  maximum time:     58.159 ms (87.54% GC)
  --------------
  samples:          643
  evals/sample:     1

In [35]:
const p2 = 2
function pow_array2(x::Vector{Float64})
    s = 0.0
    for y in x
        s = s + y^p2
    end
    return s
end


pow_array2 (generic function with 1 method)

In [36]:
@benchmark pow_array2(t)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     81.326 μs (0.00% GC)
  median time:      81.336 μs (0.00% GC)
  mean time:        83.409 μs (0.00% GC)
  maximum time:     461.133 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [38]:
trunc(x) = x < 0 ? zero(x) : x

trunc (generic function with 1 method)

In [39]:
function sqrt_sin(x)
    y = trunc(x)
    return sin(sqrt(y) + 1)
end


sqrt_sin (generic function with 1 method)

In [40]:
@code_typed sqrt_sin(1)

CodeInfo(
1 ─ %1  = (Base.slt_int)(x, 0)::Bool
└──       goto #3 if not %1
2 ─       goto #4
3 ─       goto #4
4 ┄ %5  = φ (#2 => 0, #3 => _2)::Int64
│   %6  = (Base.sitofp)(Float64, %5)::Float64
│   %7  = (Base.lt_float)(%6, 0.0)::Bool
└──       goto #6 if not %7
5 ─       invoke Base.Math.throw_complex_domainerror(:sqrt::Symbol, %6::Float64)::Union{}
└──       $(Expr(:unreachable))::Union{}
6 ┄ %11 = (Base.Math.sqrt_llvm)(%6)::Float64
└──       goto #7
7 ─       goto #8
8 ─ %14 = (Base.add_float)(%11, 1.0)::Float64
│   %15 = invoke Main.sin(%14::Float64)::Float64
└──       return %15
) => Float64